In [6]:
import pandas as pd
import numpy as np
import movie_stars
import warnings
from astropy.io import fits
import yaml
warnings.filterwarnings("ignore")

time_to_integrate = np.linspace(0., 50., 100)

# apogee catalog to crossmatch with
df_apogee = pd.read_csv('/Users/cam/Downloads/apogee_dr17.csv')
age_min = 35
age_max = 60

points_disappear = False

# Sebastian Sco-Cen

In [7]:
#df_sc = pd.read_csv('/Users/cam/Downloads/sc_sebastian_apogee_and_sos.csv')
df_sc = pd.read_csv('/Users/cam/Downloads/sc_sebastian_full_new.csv')
df_sc = df_sc.loc[df_sc.n_rv > 10]

# creating movie class
sco_cen_key_dict = {
    'ra': 'ra',
    'dec': 'dec',
    'parallax': 'parallax',
    'pmra': 'pmra',
    'pmdec': 'pmdec',
    'rv_chosen': 'radial_velocity',
    'label': 'group_name',
    'age': 'age'
}
sco_cen_marker_props = {
    'size': 5,
    'opacity': 1.,
    'symbol': 'age-based',
    'color': '#00FA9A',
    'line': dict(width=0.)
}

sco_cen = movie_stars.MovieStar(name='SiGMA Sco-Cen',
                                time_to_integrate=time_to_integrate,
                                df=df_sc,
                                key_dict=sco_cen_key_dict,
                                marker_properties=sco_cen_marker_props,
                                disappear=points_disappear)

sco_cen.marker_properties['symbol'] == 'age-based'
sco_cen.df_integrated.to_csv('../clusters_integrated/sco_cen.csv', index=False)


# Kerr+2021 Groups (besides Sco-Cen)

In [8]:
#df_kerr = pd.read_csv('/Users/cam/Downloads/kerr_xmatched.csv')
df_kerr = pd.read_csv('../kerr_sos_edr3.csv')
df_kerr = df_kerr.rename(columns={'EOM': 'eom'})
#(df_kerr.tlc != 22)
#df_kerr = df_kerr.loc[(df_kerr.eom >= 0)]
df_kerr = pd.merge(left=df_kerr,
                   right=df_apogee,
                   how='left',
                   left_on='source_id',
                   right_on='gaiaedr3_source_id')
df_kerr = df_kerr.loc[(df_kerr.parallax > 0.)]

df_kerr_apogee = df_kerr.loc[df_kerr['vhelio_avg'].notnull()]
df_kerr_apogee['rv_best'] = df_kerr_apogee['vhelio_avg']
df_kerr_apogee['rv_best_err'] = df_kerr_apogee['verr']

# df_kerr_gaia = df_kerr.loc[(df_kerr['vhelio_avg'].isnull()) & (df_kerr['radial_velocity'].notnull())]
# df_kerr_gaia['rv_best'] = df_kerr_gaia['radial_velocity']

df_kerr_sos = df_kerr.loc[(df_kerr['RVcor_merged'].notnull())
                          & (df_kerr['vhelio_avg'].isnull())]
df_kerr_sos['rv_best'] = df_kerr_sos['RVcor_merged']
df_kerr_sos['rv_best_err'] = df_kerr_sos['ERRVcor_merged']

df_kerr = pd.concat([df_kerr_apogee, df_kerr_sos])
df_kerr['rv_best_rel_err'] = np.abs(df_kerr['rv_best_err'].values /
                                    df_kerr['rv_best'].values)
df_kerr = df_kerr.loc[df_kerr['rv_best_rel_err'] < .3]

#df_kerr = df_kerr.drop(columns = ['radial_velocity'])

df_kerr = df_kerr.groupby(['eom']).median().reset_index()
df_kerr = df_kerr.loc[df_kerr.Age.notnull()]

kerr_key_dict = {
    'ra': 'ra',
    'dec': 'dec',
    'parallax': 'parallax',
    'pmra': 'pmra',
    'pmdec': 'pmdec',
    'rv_best': 'radial_velocity',
    'eom': 'group_name',
    'Age': 'age'
}
kerr_marker_props = {
    'size': 5,
    'opacity': 1.,
    'symbol': 'age-based',
    'color': '#4169E1',
    'line': dict(width=0.)
}

kerr = movie_stars.MovieStar(name='Kerr+2021',
                             time_to_integrate=time_to_integrate,
                             df=df_kerr,
                             key_dict=kerr_key_dict,
                             marker_properties=kerr_marker_props,
                             disappear=points_disappear)
kerr.df_integrated.to_csv('../clusters_integrated/kerr.csv', index=False)


# Nuria Groups

In [9]:
df_nuria = pd.read_csv('/Users/cam/Downloads/table_YLAs_equ_family2.csv')
df_nuria['age'] = np.ones(len(df_nuria)) * 35

nuria_key_dict = {
    'ra': 'ra',
    'dec': 'dec',
    'parallax': 'parallax',
    'pmra': 'pmra',
    'pmdec': 'pmdec',
    'RV': 'radial_velocity',
    'age': 'age',
    'Group': 'group_name'
}
nuria_marker_props = {
    'size': 5,
    'color': 'orange',
    'opacity': 1.,
    'symbol': 'age-based',
    'line': dict(width=0.)
}

nuria = movie_stars.MovieStar(name='Nuria Moving Groups',
                              time_to_integrate=time_to_integrate,
                              df=df_nuria,
                              key_dict=nuria_key_dict,
                              marker_properties=nuria_marker_props, disappear=points_disappear)
nuria.df_integrated.to_csv('../clusters_integrated/nuria.csv', index=False)


# Swiggum/Chen Orion

In [10]:
df_orion = pd.read_csv(
    '/Users/cam/Desktop/astro_research/orion/orion_full_groups_with_age.csv')
df_orion = df_orion.loc[df_orion.apogee2_radial_velocity.notnull()]
df_orion = df_orion.groupby(['label']).median().reset_index()
df_orion = df_orion.loc[df_orion.probability > .2]

orion_key_dict = {
    'ra': 'ra',
    'dec': 'dec',
    'parallax': 'parallax',
    'pmra': 'pmra',
    'pmdec': 'pmdec',
    'apogee2_radial_velocity': 'radial_velocity',
    'age': 'age',
    'label': 'group_name'
}
orion_marker_props = {
    'size': 7,
    'color': '#00BFFF',
    'opacity': .7,
    'symbol': 'age-based',
    'line': dict(width=0.)
}
orion = movie_stars.MovieStar(name='Swiggum+21 Orion',
                              time_to_integrate=time_to_integrate,
                              df=df_orion,
                              key_dict=orion_key_dict,
                              marker_properties=orion_marker_props,
                              disappear=points_disappear)
orion.df_integrated.to_csv('../clusters_integrated/orion.csv', index=False)


# Maria Catalog

In [11]:
#df_maria_rw = pd.read_csv('/Users/cam/Downloads/maria_rw_clusters.csv')
df_maria_rw = pd.read_csv('/Users/cam/Downloads/RW_clusters_selection4.csv')
# df_maria_rw = df_maria_rw.loc[(df_maria_rw.rv_weighted.notnull()) &
#                               (df_maria_rw.rv_weighted != 0.) &
#                               (df_maria_rw.N_rv_weighted > 5) &
#                               (df_maria_rw.rv_weighted.abs().between(-50, 50))]
df_maria_rw['age_myr'] = (10**df_maria_rw.age.values)/1e6
df_maria_rw = df_maria_rw.drop(columns = ['age'])
#df_maria_rw = df_maria_rw.drop(columns=['radial_velocity', 'age'])
#df_maria_rw.to_csv('/Users/cam/Downloads/maria_catalog_rw.csv', index = False)
maria_clusters_key_dict = {
    'ra': 'ra',
    'dec': 'dec',
    'parallax': 'parallax',
    'pmra': 'pmra',
    'pmdec': 'pmdec',
    'rv_weighted_med': 'radial_velocity',
    'age_myr': 'age',
    'cluster': 'group_name'
}

maria_marker_props = {
    'size': 11.,
    'color': '#FF4500',
    'opacity': .7,
    'symbol': 'age-based',
    'line': dict(width=0.)
}

maria_clusters_rw = movie_stars.MovieStar(name='Radcliffe Wave',
                                          df=df_maria_rw,
                                          key_dict=maria_clusters_key_dict,
                                          time_to_integrate=time_to_integrate,
                                          marker_properties=maria_marker_props,
                                          disappear=points_disappear,
                                          visible=True)
# maria_clusters_rw.df_integrated.to_csv(
#     '../clusters_integrated/maria_clusters_rw.csv', index=False)

In [12]:
df_maria = pd.read_csv(
    '/Users/cam/Downloads/maria_catalog_september_with_vlsr.csv')
ntot = df_maria.N_tot.values
#df_maria = df_maria.drop(columns=['radial_velocity', 'age'])
df_maria = df_maria.sample(frac=1).reset_index(drop=True)
age_cut_upper = 60
age_cut_lower = 25
df_maria = df_maria.loc[df_maria.ref != 'Liu+2019']

df_maria = df_maria.loc[(df_maria.rv_weighted.notnull())
                        & (df_maria.rv_weighted != 0.) &
                        (df_maria.N_rv_weighted > 5) &
                        (df_maria.rv_weighted.abs().between(-100, 100))]

df_maria_older = df_maria.loc[(df_maria.age_myr <= age_cut_upper)
                              & (df_maria.age_myr > age_cut_lower)]
df_maria_older.to_csv('/Users/cam/Downloads/maria_older.csv', index = False)
maria_marker_props = {
    'size': 5.,
    'color': 'cyan',
    'opacity': 1.,
    'symbol': 'age-based',
    'line': dict(width=0.)
}

maria_clusters = movie_stars.MovieStar(
    name='Maria Catalog: {} Myr < Age <= {} Myr'.format(
        age_cut_lower, age_cut_upper),
    df=df_maria_older,
    key_dict=maria_clusters_key_dict,
    time_to_integrate=time_to_integrate,
    marker_properties=maria_marker_props,
    disappear=points_disappear,
    visible=True)
# maria_clusters.df_integrated.to_csv(
#     '../clusters_integrated/maria_clusters.csv', index=False)

In [13]:
df_maria_young = df_maria.loc[df_maria.age_myr <= age_cut_lower]
df_maria_young.to_csv('/Users/cam/Downloads/maria_young.csv', index = False)
maria_marker_props_copy = maria_marker_props.copy()
maria_marker_props_copy['size'] = 5.
maria_marker_props_copy['color'] = 'red'
maria_clusters_young = movie_stars.MovieStar(name='Maria Catalog: Age <= {} Myr'.format(age_cut_lower),
                                       df=df_maria_young,
                                       key_dict=maria_clusters_key_dict,
                                       time_to_integrate=time_to_integrate,
                                       marker_properties=maria_marker_props_copy,
                                       disappear=points_disappear,
                                       visible=True)

In [10]:
# highlighted clusters

names = [
    'ASCC_32', 'OC-0450', 'CWNU_1162', 'CWNU_1169', 'UPK_473', 'UPK_421',
    'CWNU_1178', 'CWNU_1206', 'CWNU_1162', 'CWNU_1178', 'CWNU_1204',
    'Gulliver_10', 'CWNU_1159', '2288', 'OC-0411', 'OC-0399', 'OC-0410',
    'OC-0407', 'CWNU_1013', 'OC-0408', 'UPK_500', '2395', '2271', 'IC_2395',
    'UPK_523', '2388', 'Collinder_132', 'UPK_477', 'UBC_7', 'UPK_469',
    'UPK_482', 'NGC_2451B', 'UPK_540', 'CWNU_1024', '2436', '2386',
    'CWNU_1046', 'UPK_478', 'LP_2383', '2383', 'Theia_35', 'UPK_451', '2397',
    'Collinder_135', 'Collinder_140', 'Ruprecht_31', 'CWNU_1101', 'NGC_2547',
    'CWNU_1101', 'CWNU_1082', 'UPK_514', 'CWNU_528', '2400', 'UPK_496',
    'UPK_535', 'CWNU_1144', 'UPK_489', 'UPK_471', 'Theia_97', '2271',
    'OC-0395', 'OC-0406', 'NGC_2362', 'LP_2270'
]
feature_name = 'Expanding Clusters'

#names = ['NGC_2547']
#feature_name = 'NGC_2547'

df_maria_fb = df_maria.loc[df_maria.cluster.isin(names)]
df_maria_fb.to_csv('/Users/cam/Downloads/maria_clusters_feedback.csv', index=False)
maria_marker_props_fb = {
    'size': 5.,
    'color': '#7B68EE',
    'opacity': 1.,
    'symbol': 'age-based',
    'line': dict(width=0.)
}

maria_clusters_fb = movie_stars.MovieStar(
    name=feature_name,
    df=df_maria_fb,
    key_dict=maria_clusters_key_dict,
    time_to_integrate=time_to_integrate,
    marker_properties=maria_marker_props_fb,
    disappear=points_disappear,
    visible=True)

In [11]:
names2 = [
    'Alessi_5', 'UPK_502', 'NGC_3228', 'CWNU_1085', 'ASCC_58', 'Platais_9',
    'UPK_507', 'UPK_481', 'UPK_554', 'UPK_538', 'UPK_479', 'UPK_438',
    'UPK_481', 'Teutsch_38', 'CWNU_1198', 'CWNU_1037', 'CWNU_1039', 'Theia_58',
    'UPK_488', 'CWNU_1102', 'BH_23', 'UPK_515', 'Theia_246', 'CWNU_1016',
    'CWNU_280', 'Trumpler_10', 'UPK_511', 'CWNU_1034', 'CWNU_1044', 'CWNU_515',
    'NGC_2451A', 'UPK_449', 'CWNU_1020', 'OC-0423', 'CWNU_1069', 'IC_2391',
    'CWNU_1134', 'CWNU_1160', 'CWNU_1134', 'CWNU_1078', 'CWNU_1167',
    'CWNU_289', 'BH_164', 'UPK_600', 'CWNU_1078', 'UPK_569', 'CWNU_1008',
    'CWNU_1019', 'UPK_569', 'CWNU_1173', 'CWNU_1016'
]
feature_name2 = 'Maria Catalog: Radially migrating clusters'

#names = ['NGC_2547']
#feature_name = 'NGC_2547'

df_maria_fb2 = df_maria.loc[df_maria.cluster.isin(names2)]

maria_marker_props_fb2 = {
    'size': 5.,
    'color': '#7FFFD4',
    'opacity': 1.,
    'symbol': 'age-based',
    'line': dict(width=0.)
}

maria_clusters_fb2 = movie_stars.MovieStar(
    name=feature_name2,
    df=df_maria_fb2,
    key_dict=maria_clusters_key_dict,
    time_to_integrate=time_to_integrate,
    marker_properties=maria_marker_props_fb2,
    disappear=points_disappear,
    visible=True)


In [16]:
# names_split = [
#     'UPK_648', 'OC-0690', 'CWNU_1261', 'Upper_Sco', 'CWNU_1226', 'CWNU_1146',
#     'CWNU_1261', 'CWNU_1004', 'OC-0666',  'PHOC_39',
#     'farSouth', 'Distrib', 'Main', 'UPK_39', 'UPK_41', 'OC-0062', 'UPK_38',
#     'UPK_36'
# ]
df_maria_split = pd.read_csv('/Users/cam/Downloads/maria_young_split_names')
df_maria_split.to_csv('/Users/cam/Desktop/astro_research/radcliffe/galactic_motions_plotly/data/maria_clusters_split.csv', index = False)
#df_maria_split = df_maria.loc[df_maria.cluster.isin(names_split)]

maria_marker_props_split = {
    'size': 11.,
    'color': '#7FFFD4',
    'opacity': 1.,
    'symbol': 'age-based',
    'line': dict(width=0.)
}

maria_clusters_split = movie_stars.MovieStar(
    name='Split',
    df=df_maria_split,
    key_dict=maria_clusters_key_dict,
    time_to_integrate=time_to_integrate,
    marker_properties=maria_marker_props_split,
    disappear=points_disappear,
    visible=True)


In [13]:
df_maria_cepheus = pd.read_csv('/Users/cam/Downloads/maria_cepheus.csv')

maria_marker_props_cepheus = {
    'size': 11.,
    'color': '#87CEFA',
    'opacity': 1.,
    'symbol': 'age-based',
    'line': dict(width=0.)
}

maria_clusters_cepheus = movie_stars.MovieStar(
    name='Cepheus Spur',
    df=df_maria_cepheus,
    key_dict=maria_clusters_key_dict,
    time_to_integrate=time_to_integrate,
    marker_properties=maria_marker_props_cepheus,
    disappear=points_disappear,
    visible=True)


# Laura Groups

In [14]:
df_laura = pd.read_csv('/Users/cam/Downloads/CamerenTable_CrASubgroups.csv')

laura_marker_props = {
    'size': 5.,
    'color': '#B2AC88',
    'opacity': .6,
    'symbol': 'age-based',
    'line': dict(width=0.)
}

laura_clusters_key_dict = {
    'ra': 'ra',
    'dec': 'dec',
    'parallax': 'parallax',
    'pmra': 'pmra',
    'pmdec': 'pmdec',
    'radial_velocity': 'radial_velocity',
    'age': 'age',
    'name': 'group_name'
}

laura_clusters = movie_stars.MovieStar(name='Laura Corona Australis',
                                       df=df_laura,
                                       key_dict=laura_clusters_key_dict,
                                       time_to_integrate=time_to_integrate,
                                       marker_properties=laura_marker_props,
                                       disappear=points_disappear)


# Quillen 2020 Clusters

In [15]:
#df_elias = pd.read_csv('/Users/cam/Downloads/elias_converted.csv')
df_elias = pd.read_csv('/Users/cam/Downloads/quillen.csv')
elias_props = {
    'size': 5.,
    'color': '#66CDAA',
    'opacity': 1.,
    'symbol': 'age-based',
    'line': dict(width=0.)
}
elias_clusters_key_dict = {
    'ra': 'ra',
    'dec': 'dec',
    'parallax': 'parallax',
    'pmra': 'pmra',
    'pmdec': 'pmdec',
    'radial_velocity': 'radial_velocity',
    'age_myr': 'age',
    'Name': 'group_name'
}

elias = movie_stars.MovieStar(name='Quillen+2020 OCs',
                              df=df_elias,
                              key_dict=elias_clusters_key_dict,
                              time_to_integrate=time_to_integrate,
                              marker_properties=elias_props,
                              disappear=points_disappear)


# Cantat-Gaudin+2019a Vela Region

In [16]:
df_cg_vela = pd.read_csv('/Users/cam/Downloads/cantat_2019a_vela_grouped.csv')
#df_cg_vela = pd.read_csv('/Users/cam/Downloads/cantat_2019a_vela.csv')
#df_cg_vela = df_cg_vela.loc[df_cg_vela.RV.notnull()]
cg_vela_props = {
    'size': 5.,
    'color': '#FFB6C1',
    'opacity': 1.,
    'symbol': 'age-based',
    'line': dict(width=0.)
}
cg_vela_clusters_key_dict = {
    'RA_ICRS': 'ra',
    'DE_ICRS': 'dec',
    'Plx': 'parallax',
    'pmRA': 'pmra',
    'pmDE': 'pmdec',
    'RV': 'radial_velocity',
    'age_myr': 'age',
    'Pop': 'group_name'
}

cg_vela = movie_stars.MovieStar(name='CG+2019a Vela',
                                df=df_cg_vela,
                                key_dict=cg_vela_clusters_key_dict,
                                time_to_integrate=time_to_integrate,
                                marker_properties=cg_vela_props,
                                disappear=points_disappear)


In [17]:
# make movie
# data_list = [
#     maria_clusters, maria_clusters_young, maria_clusters_rw, maria_clusters_fb, maria_clusters_fb2, sco_cen, kerr, orion,
#     laura_clusters, nuria, elias, cg_vela
# ]
#data_list = [maria_clusters, maria_clusters_young, maria_clusters_rw, maria_clusters_fb, orion, elias, cg_vela]
#data_list = [maria_clusters, maria_clusters_young, maria_clusters_rw, maria_clusters_fb, orion, sco_cen, cg_vela, elias] 
data_list = [maria_clusters, maria_clusters_young, maria_clusters_rw, maria_clusters_split, maria_clusters_cepheus, maria_clusters_fb] 

annotations = [dict(x=-272, y=120, z=50, text='Radcliffe Wave', xanchor='center', opacity=1., font=dict(size=10, color='red'), showarrow=False),
               dict(x=235, y=77, z=44, text='The Split', xanchor='center',
                    opacity=.8, font=dict(size=10), showarrow=False),
               dict(x=2144, y=-5768, z=60, text='Compact Group (t = -26.5)',
                    xanchor='center', opacity=1., font=dict(size=8), showarrow=True)
               ]


# movie_save_path = '/Users/cam/Desktop/astro_research/radcliffe/movies/gal_mov_large.html'
# movie = movie_stars.Movie(
#     movie_stars=data_list, time=time_to_integrate, movie_save_path=movie_save_path,
#     center_star='Sun', annotations=None, xyz_ranges=[25000, 25000, 400], camera_follow = False, center_galactic = True, add_extra_traces = True
# )



movie_save_path = '/Users/cam/Desktop/astro_research/radcliffe/movies/gal_mov.html'
movie = movie_stars.Movie(movie_stars=data_list,
                          time=time_to_integrate,
                          movie_save_path=movie_save_path,
                          center_star='Sun',
                          annotations=None,
                          xyz_ranges=[4000, 4000, 700],
                          camera_follow=True,
                          center_galactic=False,
                          add_extra_traces=True)
movie.make_movie()


Movie Created!


In [18]:
a= np.linspace(-60,60,400)
len(a)/2

200.0